# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846901623799                   -0.70    4.8         
  2   -7.852330301237       -2.27       -1.53    1.0   24.9ms
  3   -7.852616321815       -3.54       -2.56    1.5   27.5ms
  4   -7.852646040902       -4.53       -2.89    2.8   36.1ms
  5   -7.852646519071       -6.32       -3.19    1.0   25.5ms
  6   -7.852646678880       -6.80       -4.00    1.0   25.4ms
  7   -7.852646686476       -8.12       -5.20    1.8   30.4ms
  8   -7.852646686725       -9.60       -5.52    2.5   34.9ms
  9   -7.852646686726      -12.08       -5.54    1.2   27.4ms
 10   -7.852646686730      -11.42       -6.41    1.0   25.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846852144716                   -0.70           4.5         
  2   -7.852551715056       -2.24       -1.63   0.80    2.0    265ms
  3   -7.852639117674       -4.06       -2.73   0.80    1.0   23.6ms
  4   -7.852646441549       -5.14       -3.35   0.80    2.0   29.5ms
  5   -7.852646679503       -6.62       -4.22   0.80    1.8   27.8ms
  6   -7.852646686604       -8.15       -4.73   0.80    1.8   28.5ms
  7   -7.852646686723       -9.92       -5.63   0.80    1.2   25.4ms
  8   -7.852646686730      -11.18       -6.46   0.80    2.2   30.3ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.386914e+01     3.332447e+00
 * time: 0.06284904479980469
     1     1.651822e+00     1.738108e+00
 * time: 0.3018620014190674
     2    -1.434580e+00     2.123755e+00
 * time: 0.3269619941711426
     3    -3.703527e+00     1.795098e+00
 * time: 0.36247992515563965
     4    -5.182449e+00     1.538131e+00
 * time: 0.39812397956848145
     5    -6.857246e+00     9.379068e-01
 * time: 0.43375301361083984
     6    -7.002092e+00     1.434045e+00
 * time: 0.45844388008117676
     7    -7.437888e+00     1.181109e+00
 * time: 0.4830329418182373
     8    -7.691849e+00     4.500031e-01
 * time: 0.5183999538421631
     9    -7.755886e+00     4.329814e-01
 * time: 0.5430300235748291
    10    -7.812780e+00     8.410993e-02
 * time: 0.5676958560943604
    11    -7.830640e+00     1.599639e-01
 * time: 0.592555046081543
    12    -7.842609e+00     7.713079e-02
 * time: 0.6176149845123291
    13    -7.847863e+00     3.867004e-02
 * time: 0.642530

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846844717866                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645901684                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.13s
  3   -7.852646686730      -13.27       -7.24    177ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.87710249636593e-7
|ρ_newton - ρ_scfv| = 1.1016399318868204e-7
|ρ_newton - ρ_dm|   = 7.401094716846793e-10
